# Start Backend and Frontend (Background)
Run the repository `start` script to launch the FastAPI backend and nginx/frontend in the background, then display health and useful links.


In [ ]:
import os, subprocess, time, socket, pathlib
from IPython.display import display, HTML, Markdown

# Paths and env
ROOT = pathlib.Path.cwd()
START = ROOT / "start"
HOME = pathlib.Path(os.environ.get("HOME", str(ROOT)))

# Ensure executable
START.chmod(0o755)

# Spawn start in background so the notebook remains interactive
# Use setsid to detach so it keeps running if this cell finishes
proc = subprocess.Popen(["/bin/bash", "-lc", str(START)], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, start_new_session=True)

# Small wait to let services begin
time.sleep(2)

# Probe ports
backend_port = 8001
frontend_port = 8080

def wait_port(port, timeout=30):
    end = time.time() + timeout
    while time.time() < end:
        s = socket.socket()
        s.settimeout(1)
        try:
            s.connect(("127.0.0.1", port))
            s.close()
            return True
        except Exception:
            time.sleep(0.5)
    return False

b_ready = wait_port(backend_port, timeout=40)
f_ready = wait_port(frontend_port, timeout=40)

# Build correct proxied URLs that respect the JupyterHub base path
# Prefer environment-provided base, fallback to relative paths

def proxy_url(port: int, path: str = "") -> str:
    base = (
        os.environ.get("JUPYTERHUB_SERVICE_PREFIX")
        or os.environ.get("JUPYTER_SERVER_URL")
        or os.environ.get("JUPYTER_SERVER_ROOT")
        or ""
    )
    if base and not base.endswith("/"):
        base += "/"
    rel = f"proxy/{port}/" + (path.lstrip("/") if path else "")
    return base + rel if base else rel

backend_href = proxy_url(backend_port, "api/docs")
frontend_href = proxy_url(frontend_port)

html = f"""
<div style='font-family: system-ui, sans-serif;'>
  <h3>Services started (background)</h3>
  <ul>
    <li>Backend: <b style='color:{'green' if b_ready else 'red'}'>{'READY' if b_ready else 'NOT READY'}</b> — <a href='{backend_href}' target='_blank'>{backend_href}</a></li>
    <li>Frontend: <b style='color:{'green' if f_ready else 'red'}'>{'READY' if f_ready else 'NOT READY'}</b> — <a href='{frontend_href}' target='_blank'>{frontend_href}</a></li>
  </ul>
  <p>Process started with PID {proc.pid}. Check logs at <code>$HOME/uvicorn.log</code> and <code>$HOME/nginx/logs/error.log</code>.</p>
</div>
"""

display(HTML(html))


In [ ]:
from IPython.display import Javascript, Markdown, display
import os

base = (
    os.environ.get("JUPYTERHUB_SERVICE_PREFIX")
    or os.environ.get("JUPYTER_SERVER_URL")
    or os.environ.get("JUPYTER_SERVER_ROOT")
    or ""
)
if base and not base.endswith("/"):
    base += "/"

url = f"{base}proxy/{8080}/"

display(Javascript(f"window.open('{url}', '_blank');"))
display(Markdown(f"If popup was blocked, click: [Open Frontend]({url})"))
